In [218]:
import pandas as pd
import numpy as np

import seaborn as sns
from matplotlib import pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import mutual_info_score
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LogisticRegression

In [219]:
def printest(args, value):
    return print( "{} : \n {} \n".format(args, value) )

# Initial Data Preparation

Churn prediction is about identifying customers who are likely to cancel their contracts soon. If the company can do that, it can offer discounts on these services in an effort to keep the users. Here we use the dataset of churn prediction for a telecom company.

- A value of 0 indicates that the customer did not churn (they stayed with the service).
- A value of 1 indicates that the customer did churn (they left the service).

In [220]:
df = pd.read_csv('Data/WA_Fn-UseC_-Telco-Customer-Churn.csv')
df.head(2)

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,...,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,7590-VHVEG,Female,0,Yes,No,1,No,No phone service,DSL,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No
1,5575-GNVDE,Male,0,No,No,34,Yes,No,DSL,Yes,...,Yes,No,No,No,One year,No,Mailed check,56.95,1889.5,No


In [221]:
df.head(1).T

,0
customerID,7590-VHVEG
gender,Female
SeniorCitizen,0
Partner,Yes
Dependents,No
tenure,1
PhoneService,No
MultipleLines,No phone service
InternetService,DSL
OnlineSecurity,No


We see that the dataset has a few columns:
- CustomerID: the ID of the customer
- Gender: male/female
- SeniorCitizen: whether the customer is a senior citizen (0/1)
- Partner: whether they live with a partner (yes/no)
- Dependents: whether they have dependents (yes/no)
- Tenure: number of months since the start of the contract
- PhoneService: whether they have phone service (yes/no)
- MultipleLines: whether they have multiple phone lines (yes/no/no phone service)
- InternetService: the type of internet service (no/fiber/optic)
- OnlineSecurity: if online security is enabled (yes/no/no internet)
- OnlineBackup: if online backup service is enabled (yes/no/no internet)
- DeviceProtection: if the device protection service is enabled (yes/no/no internet)
- TechSupport: if the customer has tech support (yes/no/no internet)
- StreamingTV: if the TV streaming service is enabled (yes/no/no internet)
- StreamingMovies: if the movie streaming service is enabled (yes/no/no internet)
- Contract: the type of contract (monthly/yearly/two years)
- PaperlessBilling: if the billing is paperless (yes/no)
- PaymentMethod: payment method (electronic check, mailed check, bank transfer,
credit card)
- MonthlyCharges: the amount charged monthly (numeric)
- TotalCharges: the total amount charged (numeric)
- Churn: if the client has canceled the contract (yes/no)

When import a CSV file, Pandas tries to guess the right type for each column. But sometimes, it doesn't get it right. So, it's a good idea to double-check the types using ``df.dtypes``.

In [222]:
df.dtypes

customerID           object
gender               object
SeniorCitizen         int64
Partner              object
Dependents           object
tenure                int64
PhoneService         object
MultipleLines        object
InternetService      object
OnlineSecurity       object
OnlineBackup         object
DeviceProtection     object
TechSupport          object
StreamingTV          object
StreamingMovies      object
Contract             object
PaperlessBilling     object
PaymentMethod        object
MonthlyCharges      float64
TotalCharges         object
Churn                object
dtype: object

We observe that the 'TotalCharges' column poses an issue. Rather than being classified as a numeric type, such as float or integer, pandas incorrectly infers it as an object type.    

In [223]:
# Convert 'TotalCharges' to numeric, replace non-numeric with NaN
df['TotalCharges'] = pd.to_numeric(df.TotalCharges, errors='coerce')

# Create a filter for NaN values
filter = df['TotalCharges'].isna()

# Display the rows where 'TotalCharges' was NaN
print('Before:')
display(df[filter][['customerID','TotalCharges']].head(2))

# Fill NaN values with zero
df['TotalCharges'] = df['TotalCharges'].fillna(0)

# Display the rows where 'TotalCharges' was NaN before the fillna operation
print('After:')
display(df[filter][['customerID','TotalCharges']].head(2))

Before:


,customerID,TotalCharges
488,4472-LVYGI,NaN
753,3115-CZMZD,NaN


After:


,customerID,TotalCharges
488,4472-LVYGI,0.0
753,3115-CZMZD,0.0


In [224]:
# Columns

# lowering columns name and replace spaces by _
df_columns_lower = df.columns.str.lower()
df.columns  = df_columns_lower.str.replace(' ', '_')

# Rows

# boolean mask for columns with strings
column_mask = df.dtypes == 'object' 
string_columns = list(df.dtypes[column_mask].index)

# lowering rows strings and replace spaces by _
for col in string_columns:
    df[col] = df[col].str.lower().str.replace(' ', '_')

df.head(2)

,customerid,gender,seniorcitizen,partner,dependents,tenure,phoneservice,multiplelines,internetservice,onlinesecurity,...,deviceprotection,techsupport,streamingtv,streamingmovies,contract,paperlessbilling,paymentmethod,monthlycharges,totalcharges,churn
0,7590-vhveg,female,0,yes,no,1,no,no_phone_service,dsl,no,...,no,no,no,no,month-to-month,yes,electronic_check,29.85,29.85,no
1,5575-gnvde,male,0,no,no,34,yes,no,dsl,yes,...,yes,no,no,no,one_year,no,mailed_check,56.95,1889.50,no


We the that some columns has 'yes' or 'no' string, that we can convert to boolean. First consider the target variable churn

In [225]:
df.churn = (df.churn == 'yes').astype(int)

df[['customerid', 'churn']].head().T

,0,1,2,3,4
customerid,7590-vhveg,5575-gnvde,3668-qpybk,7795-cfocw,9237-hqitu
churn,0,0,1,0,1


In [226]:
df_train_full, df_test = train_test_split(df, test_size=0.2, random_state=1)
df_train, df_val = train_test_split(df_train_full, test_size=0.33, random_state=11)

y_train = df_train['churn'].values
y_val = df_val['churn'].values

del df_train['churn']
del df_val['churn']

display(df_train.head(2))

,customerid,gender,seniorcitizen,partner,dependents,tenure,phoneservice,multiplelines,internetservice,onlinesecurity,onlinebackup,deviceprotection,techsupport,streamingtv,streamingmovies,contract,paperlessbilling,paymentmethod,monthlycharges,totalcharges
2935,9435-jmlsx,male,0,yes,no,71,yes,no,dsl,yes,yes,yes,yes,yes,yes,two_year,yes,bank_transfer_(automatic),86.1,6045.9
3639,0512-flfdw,female,1,yes,no,60,yes,yes,fiber_optic,no,no,yes,no,yes,yes,one_year,yes,credit_card_(automatic),100.5,6029.0


# Exploratory Data Analysis (EDA)

We have already found a problem with the TotalCharges column and replaced the missing values with zeros. Now let’s see if we need to perform any additional null handling:

In [227]:
df_train_full.isnull().sum()

customerid          0
gender              0
seniorcitizen       0
partner             0
dependents          0
tenure              0
phoneservice        0
multiplelines       0
internetservice     0
onlinesecurity      0
onlinebackup        0
deviceprotection    0
techsupport         0
streamingtv         0
streamingmovies     0
contract            0
paperlessbilling    0
paymentmethod       0
monthlycharges      0
totalcharges        0
churn               0
dtype: int64

Let’s check the proportion of churned users among all customers. This is the **Global Churn Rate** that refers to the overall churn rate for the entire customer base of the dataset.

For that, we need to divide the number of customers who churned by the total number of customers as follows:

In [228]:
# checking the distribution of values in the target variable
churn_stats = df_train_full['churn'].agg([pd.value_counts])

# Mean
total_values = churn_stats['value_counts'].sum()
churn_stats['global_mean'] = round(churn_stats['value_counts']/total_values, 3)

display(churn_stats)


,value_counts,global_mean
0,4113,0.73
1,1521,0.27


This gives us the proportion of churned users, or the probability that a customer will churn. As we see in, approximately 27% of the customers stopped
using our services, and the rest remained as customers. 

Also, the dataset is a imbalanced one. There were three times as many people who didn’t churn in our dataset as those who did churn.

Let's separate the dataset in categorical and numerical variables:

In [229]:
# All categorical columns except 'customerid'
categorical_mask = df_train.dtypes == 'object'
categorical = list(df_train.dtypes[categorical_mask].index)
categorical.remove('customerid') 

# Manually add 'seniorcitizen' because it's an int boolean (0 or 1)
categorical.append('seniorcitizen')
printest('categorical', categorical)

# All numerical columns except 'seniorcitizen' because it's an int boolean
numerical_mask = df_train.dtypes != 'object'
numerical = list(df_train.dtypes[numerical_mask].index)
numerical.remove('seniorcitizen')
printest('numerical', numerical)

categorical : 
 ['gender', 'partner', 'dependents', 'phoneservice', 'multiplelines', 'internetservice', 'onlinesecurity', 'onlinebackup', 'deviceprotection', 'techsupport', 'streamingtv', 'streamingmovies', 'contract', 'paperlessbilling', 'paymentmethod', 'seniorcitizen'] 

numerical : 
 ['tenure', 'monthlycharges', 'totalcharges'] 



In [230]:
#Count number of distinct elements in specified axis.
df_train_full[categorical].nunique()

gender              2
partner             2
dependents          2
phoneservice        2
multiplelines       3
internetservice     3
onlinesecurity      3
onlinebackup        3
deviceprotection    3
techsupport         3
streamingtv         3
streamingmovies     3
contract            3
paperlessbilling    2
paymentmethod       4
seniorcitizen       2
dtype: int64

## Feature importance

**Global Churn Ratio**

refers to the overall churn rate for the entire customer base of the dataset.


In [231]:
global_mean = df_train_full.churn.mean()
display(global_mean)

0.26996805111821087

**Group Churn Ratio**

The churn rate within a specific customer segment, known as the group churn rate, allows for targeted analysis. By comparing this group churn rate with the overall churn rate, we can better understand how a particular group's behavior deviates from the average customer behavior. If there's a minimal difference between the group and global churn rates, it indicates that this specific group's churn behavior is not significantly different from the overall customer base. Therefore, this group's characteristics might not be a critical factor in predicting churn.

Now, let's start our analysis with the gender variable:

In [232]:
gender_mean = df_train_full.groupby('gender')['churn'].mean()
display(gender_mean)

gender
female    0.276824
male      0.263214
Name: churn, dtype: float64

The difference between the group rates for both males and females is quite small, which indicates that knowing the gender of the customer
doesn’t help us identify whether they will churn.

Now let’s take a look at another variable: partner:

In [233]:
partner_mean = df_train_full.groupby('partner')['churn'].mean()
display(partner_mean)

partner
no     0.329809
yes    0.205033
Name: churn, dtype: float64

The churn rate for people with a partner is significantly less than the rate for the ones without a partner — 20.5% versus 33%. It means that clients with no partner are more likely to churn than the ones with a partner

**Risk Ratio**

Apart from the difference between group and global churn rates, it's also useful to calculate their ratio. In statistics, this ratio is known as the 'risk ratio,' with 'risk' referring to the chance of the event of interest occurring - in our case, churning.

$$\text{Risk} =\frac{\text{Group Rate}}{\text{Global Rate}} $$

The risk can range from zero to infinity. This gives us a clear idea of how likely members of a particular group are to churn compared to the entire customer base.

- $\text{Risk Ratio} > 1$: The group exhibits a higher churn rate compared to the overall population. This implies that members of this group are more likely to churn.

- $\text{Risk Ratio} = 1$: The group's churn rate is equivalent to that of the overall population. This means the group's likelihood to churn is average.

- $\text{Risk Ratio} < 1$: The group has a lower churn rate compared to the overall population. Customers within this group are less likely to churn.
  



In [234]:
# alternative method to groupby
df_group = df_train_full.pivot_table(values = 'churn', index = 'gender', aggfunc = np.mean)
df_group = df_group.rename(columns={'churn': 'mean'})
df_group['diff'] = df_group['mean'] - global_mean
df_group['risk'] = df_group['mean']/ global_mean
display(df_group)

# alternative method to groupby
df_group = df_train_full.pivot_table(values = 'churn', index = 'partner', aggfunc = np.mean)
df_group = df_group.rename(columns={'churn': 'mean'})
df_group['diff'] = df_group['mean'] - global_mean
df_group['risk'] = df_group['mean']/ global_mean


display(df_group)

,mean,diff,risk
gender,,,
female,0.276824,0.006856,1.025396
male,0.263214,-0.006755,0.974980


,mean,diff,risk
partner,,,
no,0.329809,0.059841,1.221659
yes,0.205033,-0.064935,0.759472


The churn rates for females
and males are not significantly different from the global churn rates, so the risks for them to churn are
low: both have risks values around 1. On the other hand, the churn rate for people with no partner is
significantly higher than average, making them risky, with the risk value of 1.22. People with partners
tend to churn less, so for them, the risk is only 0.75.

for all categorical variables:

In [235]:
for col in categorical:
    df_group = df_train_full.groupby(by=col).churn.agg(['mean'])
    df_group['diff'] = df_group['mean'] - global_mean
    df_group['risk'] = df_group['mean'] / global_mean
    display(df_group)

,mean,diff,risk
gender,,,
female,0.276824,0.006856,1.025396
male,0.263214,-0.006755,0.974980


,mean,diff,risk
partner,,,
no,0.329809,0.059841,1.221659
yes,0.205033,-0.064935,0.759472


,mean,diff,risk
dependents,,,
no,0.313760,0.043792,1.162212
yes,0.165666,-0.104302,0.613651


,mean,diff,risk
phoneservice,,,
no,0.241316,-0.028652,0.893870
yes,0.273049,0.003081,1.011412


,mean,diff,risk
multiplelines,,,
no,0.257407,-0.012561,0.953474
no_phone_service,0.241316,-0.028652,0.893870
yes,0.290742,0.020773,1.076948


...

,mean,diff,risk
streamingmovies,,,
no,0.338906,0.068938,1.255358
no_internet_service,0.077805,-0.192163,0.288201
yes,0.307273,0.037305,1.138182


,mean,diff,risk
contract,,,
month-to-month,0.431701,0.161733,1.599082
one_year,0.120573,-0.149395,0.446621
two_year,0.028274,-0.241694,0.104730


,mean,diff,risk
paperlessbilling,,,
no,0.172071,-0.097897,0.637375
yes,0.338151,0.068183,1.252560


,mean,diff,risk
paymentmethod,,,
bank_transfer_(automatic),0.168171,-0.101797,0.622928
credit_card_(automatic),0.164339,-0.105630,0.608733
electronic_check,0.455890,0.185922,1.688682
mailed_check,0.193870,-0.076098,0.718121


,mean,diff,risk
seniorcitizen,,,
0,0.242270,-0.027698,0.897403
1,0.413377,0.143409,1.531208


- Gender: Little difference in churn rates between females and males, with similar means and risks close to 1 for both groups.

- Senior Citizens: Higher churn risk at 1.53, compared to 0.89 for nonseniors.

- Partner Status: Lower churn risk for customers with a partner at 0.75, compared to 1.22 for those without.

- Phone Service Usage: Near-equal churn risk to global rate for users. Lower risk for non-users, with a risk 
below 1.

- Tech Support: Higher churn risk for clients without tech support with risk 1.55.

- Contract Length: Highest churn risk for monthly contract clients, while two-year contract clients churn very rarely.

**Entropy and Mutual Information**

In order to fully understand the concept of Mutual Information, we first will explore the concept of entropy in information theory. Let's assume we have a dataset with outcomes denoted by $Y$ and an attribute symbolized by $X$, both of which are categorical variables.

Entropy for the independent variable $X$ is defined as:

$$H(X) = -\sum_i p(x_i)\log{p(x_i)}$$

Here, $p(x_i)$ represents the probability of occurrence of a specific value $x_i$ of attribute $X$. This probability is calculated as:

$$p(x_i) =\frac{n_i}{n}$$

In this equation, $n_i$ is the total number of instances where the specific $x_i$ occurs, and $n$ is the total number of instances. We can similarly compute the entropy for the target variable $Y$.

For example, in a churn analysis, $X$ could represent the random variable for gender, with $x_i$ representing a specific gender (male or female). This formula allows us to calculate the entropy of the gender distribution in the dataset.

The next step involves connecting the entropies of $X$ and $Y$. We accomplish this by constructing a conditional entropy of $X$ given a specific target value $y_i$, formulated as:

$$H(X|y_i) = - \sum_j p(x_j|y_i)\log p(x_j|y_i) $$ 

Here, $p(x_j|y_i)$ stands for the conditional probability, computed as:

$$p(x_j|y_i) = \frac{n_{ij}}{n_i}$$

In this formula, $n_{ij}$ denotes the count of instances where a specific attribute value $x_j$ (e.g., male) and a specific target value $y_i (e.g., churn) occur together. On the other hand, $n_i$ represents the total count of instances where the specific target value $y_i$ (either churn or not churn) occurs, regardless of the attribute value.

With $X$ representing gender and $Y$ as churn, the entropy measure $H(X|y_i)$ calculates the entropy of the gender category conditional on churn $(y_1)$ and no churn $(y_0)$.

We can also express the conditional probability in terms of the joint probability, $p(x_j,y_i)$, computed as:

$$p(x_j,y_i)  = \frac{n_{ij}}{n}$$  

This allows us to express the conditional probability as:

$$p(x_j|y_i) = \frac{p(x_j,y_i)}{p(y_i)}$$

This way we can rewrite the conditional entropy as:

$$H(X|y_i) = - \sum_j \frac{p(x_j,y_i)}{p(y_i)}\log \bigg(\frac{p(x_j,y_i)}{p(y_i)}\bigg) $$ 

To calculate the entropy of variable $X$ conditional on the entire set $Y$, we need to compute the weighted sum of the conditional entropies for each possible outcome in $Y$. This can be represented mathematically as:

$$
\begin{align*}
H(X|Y) &= - \sum_i\sum_j p(y_i)H(X|y_i) \\
       &= - \sum_i\sum_j p(y_i)\bigg[\frac{p(x_j,y_i)}{p(y_i)}\log \bigg(\frac{p(x_j,y_i)}{p(y_i)}\bigg)\bigg]\\
       &= - \sum_i\sum_j p(x_j,y_i)\log \bigg(\frac{p(x_j,y_i)}{p(y_i)}\bigg)
\end{align*}
$$

This computation yields the expected conditional entropy over all possible outcomes of $Y$.

Further, the equation can be rearranged using logarithm properties to obtain:

$$
\begin{align*}
H(X|Y) &= - \sum_i\sum_j p(x_j,y_i)\log{p(x_j,y_i)} + \sum_i  \bigg(\log{p(y_i)\sum_j p(x_j,y_i)}\bigg) \\
       &= - \sum_i\sum_j p(x_j,y_i)\log{p(x_j,y_i)} + \sum_i  p(y_i)\log{p(y_i) }\\
       &= H(X,Y) - H(Y)
\end{align*}
$$

In this context, joint entropy $H(X,Y)$ quantifies the combined uncertainty of $X$ and $Y$, encapsulating the question, "How much do I not know about both $X$ and $Y$ jointly?" Conversely, conditional entropy $H(X|Y)$ expresses the remaining uncertainty of $X$ when we have some knowledge of $Y$, answering the query, "Given some knowledge of $Y$, what is my remaining uncertainty about $X$?"

Thus, the formula

$$H(X|Y) = H(X,Y) - H(Y)$$ 

is interpreted as the removal of uncertainty about $Y$ from the total joint uncertainty, leaving behind the conditional entropy $H(X|Y)$. This represents the residual uncertainty about $X$ once knowledge about $Y$ is accounted for. This interpretation underscores the inherent connection between these entropy measures, providing a clear mathematical understanding of their relationships. 

Similarly, we can define Mutual Information (MI) between two variables $X$ and $Y$ as:

$$MI(X;Y) = H(X) − H(X∣Y)$$

The Mutual Information, $MI(X;Y)$, denotes the reduction in uncertainty about $X$ due to the knowledge of $Y$, and vice versa. In other words, it quantifies the amount of information gained about $X$ after learning about $Y$, and the amount of information gained about $Y$ after learning about $X$. This is because mutual information is symmetric, i.e., $MI(X;Y) = MI(Y;X)$.

Here's how to interpret mutual information:

- **If $MI(X;Y)$ is high:** knowing $Y$ greatly reduces our uncertainty about $X$, and knowing $X$ greatly reduces our uncertainty about $Y$. This is the case where $X$ and $Y$ have some dependency.
 
- **If $MI(X;Y)$ is zero:**  knowing $Y$ does nothing to decrease our uncertainty about $X$, and knowing $X$ does nothing to decrease our uncertainty about $Y$. This is the case when $X$ and $Y$ are independent.

Therefore, Mutual Information, $MI(X;Y)$, serves as a measure of the decrease in uncertainty about $X$ (alternatively, the information about $X$ that we gain) as a result of knowing $Y$, and the decrease in uncertainty about $Y$ (or the information about $Y$ that we gain) as a result of knowing $X$.

Mutual Information can also be expressed in terms of joint and marginal probabilities as follows:

$$\textbf{MI}(X;Y) = \sum_i \sum_j P(x_i,y_j) \log \bigg(\frac{P(x_i,y_j)}{P(x_i)P(y_j)}\bigg)$$


We can apply this concept for categorical variables as follows:

In [236]:
def calculate_mi(series):
    return mutual_info_score(series, df_train_full.churn)

# Apply a function along one of the axis
df_mi = df_train_full[categorical].apply(calculate_mi)
df_mi = df_mi.sort_values(ascending=False).to_frame(name='MI')

print("Hight values of MI:")
display(df_mi.head())
print("\n Low values of MI:")
display(df_mi.tail())

Hight values of MI:


,MI
contract,0.098320
onlinesecurity,0.063085
techsupport,0.061032
internetservice,0.055868
onlinebackup,0.046923



 Low values of MI:


,MI
partner,0.009968
seniorcitizen,0.009410
multiplelines,0.000857
phoneservice,0.000229
gender,0.000117


As we see, contract, onlinesecurity, and techsupport are among the most
important features. It’s not surprising that gender is among the least important features, so we shouldn’t expect it to be useful for the model.

## Numerical variables

**Pearson's correlation coefficient**

While mutual information can effectively quantify the degree of dependency between two categorical variables, it doesn't function well when one of the variables is numerical. Hence, it isn't appropriate for the three numerical variables in our dataset.

Instead, Pearson's correlation coefficient can be used as a measure of the linear correlation or dependence between two variables, X and Y. The coefficient provides a value between $[1, −1]$, where:

- 1 is a total positive linear correlation, 
- 0 is no linear correlation
- −1 is a total negative linear correlation. 

the correlation for a pair of random variables $(X, Y)$ is given by:

$$
\rho(X,Y) = \frac{\text{cov}(X,Y)}{\sigma_X \sigma_Y}
$$

In this equation, $\text{cov}(X,Y)$ represents the covariance between X and Y, while $\sigma_X$ and $\sigma_Y$ are the standard deviations of X and Y, respectively. The covariance, a measure of how much two random variables change together, is computed as follows:

$\text{cov}(X,Y) = E[(X - E[X])(Y - E[Y])]$

Here, $E[X]$ and $E[Y]$ are the expected values (or means) of X and Y, respectively. This formula essentially captures the joint variability of X and Y.

The standard deviations, which measure the dispersion or spread of the data points from the mean, are computed as follows:

$$\sigma_X = \sqrt{E[X^2] - E[X]^2}$$
$$\sigma_Y = \sqrt{E[Y^2] - E[Y]^2}$$

Pearson's correlation coefficient can be a valuable tool to understand and visualize the linear relationships between features. This can inform feature selection, as highly correlated features often carry redundant information

Let's translate this to python code:

In [237]:
def pearson_corr(x, y):
    # Compute the means of X and Y
    mean_x = x.agg(np.mean)
    mean_y = y.agg(np.mean)
    
    mean_x_values = mean_x.values
    x_labels = mean_x.index
    x_values = x.values

    mean_y = y.agg(np.mean)
    y_values = y.values.reshape(-1,1)

    # Compute the covariance
    cov_XY = np.mean((x_values - mean_x_values) * (y_values - mean_y), axis = 0)

    # Compute the standard deviations
    std_X = np.sqrt(np.mean((x_values - mean_x_values)**2, axis = 0))
    std_Y = np.sqrt(np.mean((y_values - mean_y)**2, axis = 0))

    # Compute the correlation
    corr_XY = cov_XY / (std_X * std_Y)

    # Create the DataFrame
    corr_XY = pd.DataFrame(corr_XY, index=x_labels, columns=['pearson_correlation'])
    return corr_XY

In [238]:
corr_XY = pearson_corr(df_train_full[numerical],df_train_full['churn'] )
display(corr_XY)

,pearson_correlation
tenure,-0.351885
monthlycharges,0.196805
totalcharges,-0.196353


Using pandas to calculate the correlation:

In [239]:
df_train_full[numerical].corrwith(df_train_full.churn).to_frame('pearson_correlation')

,pearson_correlation
tenure,-0.351885
monthlycharges,0.196805
totalcharges,-0.196353


In [240]:
df_train_full.groupby(by='churn')[numerical].mean()

,tenure,monthlycharges,totalcharges
churn,,,
0,37.531972,61.176477,2548.021627
1,18.070348,74.521203,1545.689415


As we can see: 

- tenure has a high negative correlation (-0.35): as tenure grows, churn rate goes down. 

- monthlycharges has positive correlation (0.19): the more customers pay, the more likely they are to churn

# Feature engineering

## One-hot encoding

When dealing with a categorical variable like "contract" with multiple possible values (e.g., monthly, yearly, and two-year), we can use a technique called one-hot encoding to represent each value as a binary vector. In this case, if we have a customer with a yearly contract, the one-hot encoding representation would be (0, 1, 0).

This encoding scheme works by designating the "active" or "hot" value as 1, while the other values are considered "not active" or "cold" and assigned a value of 0. In our example, the yearly contract is the active value, so it receives a 1, while the monthly and two-year contracts are not active and receive 0.

In [241]:
#Transform dataframe in dict inside a list
train_dict = df_train[categorical + numerical].to_dict(orient='records')
train_dict[0]

{'gender': 'male',
 'partner': 'yes',
 'dependents': 'no',
 'phoneservice': 'yes',
 'multiplelines': 'no',
 'internetservice': 'dsl',
 'onlinesecurity': 'yes',
 'onlinebackup': 'yes',
 'deviceprotection': 'yes',
 'techsupport': 'yes',
 'streamingtv': 'yes',
 'streamingmovies': 'yes',
 'contract': 'two_year',
 'paperlessbilling': 'yes',
 'paymentmethod': 'bank_transfer_(automatic)',
 'seniorcitizen': 0,
 'tenure': 71,
 'monthlycharges': 86.1,
 'totalcharges': 6045.9}

In [242]:
categorical

['gender',
 'partner',
 'dependents',
 'phoneservice',
 'multiplelines',
 'internetservice',
 'onlinesecurity',
 'onlinebackup',
 'deviceprotection',
 'techsupport',
 'streamingtv',
 'streamingmovies',
 'contract',
 'paperlessbilling',
 'paymentmethod',
 'seniorcitizen']

In [243]:
train_dict

[{'gender': 'male',
  'partner': 'yes',
  'dependents': 'no',
  'phoneservice': 'yes',
  'multiplelines': 'no',
  'internetservice': 'dsl',
  'onlinesecurity': 'yes',
  'onlinebackup': 'yes',
  'deviceprotection': 'yes',
  'techsupport': 'yes',
  'streamingtv': 'yes',
  'streamingmovies': 'yes',
  'contract': 'two_year',
  'paperlessbilling': 'yes',
  'paymentmethod': 'bank_transfer_(automatic)',
  'seniorcitizen': 0,
  'tenure': 71,
  'monthlycharges': 86.1,
  'totalcharges': 6045.9},
 {'gender': 'female',
  'partner': 'yes',
  'dependents': 'no',
  'phoneservice': 'yes',
  'multiplelines': 'yes',
  'internetservice': 'fiber_optic',
  'onlinesecurity': 'no',
  'onlinebackup': 'no',
  'deviceprotection': 'yes',
  'techsupport': 'no',
  'streamingtv': 'yes',
  'streamingmovies': 'yes',
  'contract': 'one_year',
  'paperlessbilling': 'yes',
  'paymentmethod': 'credit_card_(automatic)',
  'seniorcitizen': 1,
  'tenure': 60,
  'monthlycharges': 100.5,
  'totalcharges': 6029.0},
 {'gender':

In [244]:
dv = DictVectorizer(sparse=False)
dv.fit(train_dict)
X_train = dv.transform(train_dict)

X_train[0].shape

(45,)

In [245]:
dv.get_feature_names_out ()

array(['contract=month-to-month', 'contract=one_year',
       'contract=two_year', 'dependents=no', 'dependents=yes',
       'deviceprotection=no', 'deviceprotection=no_internet_service',
       'deviceprotection=yes', 'gender=female', 'gender=male',
       'internetservice=dsl', 'internetservice=fiber_optic',
       'internetservice=no', 'monthlycharges', 'multiplelines=no',
       'multiplelines=no_phone_service', 'multiplelines=yes',
       'onlinebackup=no', 'onlinebackup=no_internet_service',
       'onlinebackup=yes', 'onlinesecurity=no',
       'onlinesecurity=no_internet_service', 'onlinesecurity=yes',
       'paperlessbilling=no', 'paperlessbilling=yes', 'partner=no',
       'partner=yes', 'paymentmethod=bank_transfer_(automatic)',
       'paymentmethod=credit_card_(automatic)',
       'paymentmethod=electronic_check', 'paymentmethod=mailed_check',
       'phoneservice=no', 'phoneservice=yes', 'seniorcitizen',
       'streamingmovies=no', 'streamingmovies=no_internet_service',

# Logistic Regression

Consider a hypothetical dataset, denoted as $\mathbf{D}$, consisting of multiple attributes and a target variable:

$$\mathbf{D} =
\left( \begin{array}{c|cccc|c}
~    &X_{0}&X_{1}&\cdots & X_{d}  & Y\\
\hline
x_{1} &1& x_{11}& \cdots&x_{1d}&y_1 \\
\vdots&\vdots&\vdots&\ddots&\vdots&\vdots\\
x_{n}&1&x_{n1}&\cdots&x_{nd}&y_n
\end{array} \right).$$

This matrix $\mathbf{D}$ represents our dataset. Each row corresponds to a unique customer and each column (aside from the last one) corresponds to a specific attribute or feature that describes our customers. $X_0$ is a column of ones, typically referred to as the bias or intercept term.

The target values $\mathbf{Y} = (y_1, \cdots, y_i, \cdots, y_n)$ are binary, taking on values within the set $\{0,1\}$, where $i$ ranges from $1$ to $n$. Here, $y_i=0$ denotes a customer who does not churn, while $y_i=1$ signifies a customer who churns. In other words, our goal is to predict whether a customer will churn or not based on their attributes.

In the context of logistic regression, we aim to answer the following question: Given any vector of independent attribute values of a customer $\mathbf{x}_i = (1,x_{i1}, \cdots, x_{id})^T$, what is the probability that the customer will churn, i.e., $y_i = 1$?

The vector $\mathbf{x}_i$ contains the attribute values for customer $i$ and the transpose operation $^T$ converts this row vector into a column vector, which aligns with our weight vector for the dot product in the subsequent computation

We also represent the multivariate random vector of all predictors for the customers as $\mathbf{X} = ({X}_{0}, \cdots, {X}_{d})^T$. This is a matrix that contains the attribute values of all customers in our dataset.

Since there are only two possible outcomes for $Y$, the probability mass function can be defined as:

$$P(\mathbf{Y} = 1|\mathbf{X}= \mathbf{x}_i) = \pi( \mathbf{x}_i) ~~~~~P(\mathbf{Y} = 0|\mathbf{X}= \mathbf{x}_i) = 1- \pi( \mathbf{x}_i)$$

This states that the probability of a customer churning (i.e., $Y=1$) given their attribute values $\mathbf{x}_i$ is represented by $\pi( \mathbf{x}_i)$. Conversely, the probability of them not churning (i.e., $Y=0$) is the complement of that, $1-\pi( \mathbf{x}_i)$.

Here I'm using the following notation:

$$P(Y|\mathbf{X}= \mathbf{x}_i) = P(Y = y_i = 1|{X}_{0} = 1,\cdots, X_{j} = x_{ij}, \cdots, {X}_{d}= x_{id})$$

for $j \in \{1,2,\cdots, d\}$,

This notation is a formal way to state that we're looking at the probability of $Y$ being equal to 1 given the attribute values $\mathbf{x}_i$ of a particular customer. Here $\pi( \mathbf{x}_i)$ is an unknown function of the predictors, representing the probability of $\mathbf{Y} = 1$ given a set of predictors $\mathbf{X}= \mathbf{x}_i$. Our goal in logistic regression is to estimate this function.

This function $\pi( \mathbf{x}_i)$ is essentially our logistic regression model. We need to estimate this function from our data, and it is this function that will give us the probability of a customer churning given their attribute values.

In the linear regression model, the predicted target variable for multiple attributes $\mathbf{X} = ({X}_{0}, \cdots, {X}_{d})^T$ can be represented as:

$$f(\mathbf{x}_i) = \hat{y}_i = \mathbf{w}^T\mathbf{x}_i $$

$\mathbf{w} = (w_0,w_1, \cdots, w_d)^T$ represents the weight coefficients $(w_1, \cdots, w_d)$ and bias $(w_0)$ that we learn during the training process in linear regression. These weights determine the contribution of each attribute to our prediction. The dot product of the weight vector and the attribute values gives us our predicted target value $\mathbf{\hat{y}}$.

A Naive approach would be use directly $P(\mathbf{Y} = 1|\mathbf{X}= \mathbf{x}_i) = f(\mathbf{x}_i)$, however due to the fact that $f(\mathbf{x}_i)$ can be arbitrarily large or small it's not possible for a probability. We require that the output represents a probability values, and thus we need a model that results in an output that lies in the interval $[0,1]$. For this task we rely in the sigmoid function that has values in this range, this function is also know as logistic function. the logistic function is defined as follows:

$$\theta(z) = \frac{1}{1+e^{-z}} = \frac{e^{z}}{1+e^{z}}$$

for any $z \in \mathbb{R}$ the output is $\theta(z) \in [0,1]$. An interesting property of the logistic function is that $1 - \theta(z) = \theta(-z)$:

$$1-\theta(z) = 1 - \frac{1}{1+ e^{-z}} = \frac{e^{-z}}{1+ e^{-z}} = \frac{1}{1+ e^{z}} =  \theta(-z)~~\square$$

Now we can address the probability as $P(\mathbf{Y} = 1|\mathbf{X}= \mathbf{x}_i) = \theta(f(\mathbf{x}_i))$ and define the logistic regression model as follows:


$$
\begin{align*}
P(\mathbf{Y} = 1|\mathbf{X} = \mathbf{x}_i) &= \theta(\mathbf{w}^T\mathbf{x}_i) \\
&= \frac{e^{\mathbf{w}^T\mathbf{x}_i}}{1+ e^{\mathbf{w}^T\mathbf{x}_i}}\\
\\
P(\mathbf{Y} = 0|\mathbf{X} = \mathbf{x}_i) &= 1 - P(\mathbf{Y} = 1|\mathbf{X} = \mathbf{x}_i)\\
&= \frac{1}{1 + e^{\mathbf{w}^T\mathbf{x}_i}}\\
& = \theta(-\mathbf{w}^T\mathbf{x}_i)
\end{align*}
$$

where in $P(\mathbf{Y} = 0|\mathbf{X} = \mathbf{x}_i)$ we use the property $1 - \theta(z) = \theta(-z)$. The model then can be formulated from the following probability:

$$P(\mathbf{Y}|\mathbf{X} = \mathbf{x}_i) = \theta(\mathbf{w}^T\mathbf{x}_i)^{Y}\theta(-\mathbf{w}^T\mathbf{x}_i)^{1-Y}$$

where $Y$ is a Bernoulli random variable that takes on either the values 0 or 1. We can observe that $P(\mathbf{Y}|\mathbf{X} = \mathbf{x}_i) = \theta(\mathbf{w}^T\mathbf{x}_i)^{Y}$ when $Y = 1$ and $P(\mathbf{Y}|\mathbf{X} = \mathbf{x}_i) = \theta(-\mathbf{w}^T\mathbf{x}_i)^{Y}$ when $Y = 0$.

In practice, we have a set of observed data, typically a dataset consisting of pairs $(\mathbf{x}_i, y_i)$, and our goal is to estimate the parameters $\mathbf{w}$ of our model that best fit this data. This involves maximizing the likelihood of our observed data under the model parameterized by $\mathbf{w}$:

$$\mathcal{L}(\mathbf{w};\mathbf{X},\mathbf{Y}) = P(Y|\mathbf{X}; \mathbf{w} ) = P(\mathbf{Y} = y_1,y_2, \cdots, y_n|\mathbf{X};\mathbf{w})$$

Here $P(Y|\mathbf{X}; \mathbf{w} )$ notation is used to denote the probability of $\mathbf{Y}$ given $\mathbf{X}$ under a particular model parameterized by $\mathbf{w}$. That is slight different from $P(Y|\mathbf{X})$, witch is a general  notation to denote the conditional probability of event $\mathbf{Y}$ given event $\mathbf{X}$. Under the assumption that $\mathbf{Y} = (y_1, \cdots, y_n)$ are independent given $\mathbf{X} = ({X}_{0}, \cdots, {X}_{d})^T$ , we can factorize the joint probability into a product of conditional probabilities:

$$\mathcal{L}(\mathbf{w};\mathbf{X},\mathbf{Y}) =  \prod_{i = 1}^{n} \theta(\mathbf{w}^T\mathbf{x}_i)^{y_i}\theta(-\mathbf{w}^T\mathbf{x}_i)^{1-y_i}$$

Our aim is to find the value of $\mathbf{w}$ that maximizes this likelihood, an approach known as Maximum Likelihood Estimation (MLE). In practice, we often maximize the logarithm of the likelihood (log-likelihood) to convert the product into a sum:

$$
\begin{align*}
\log(\mathcal{L}(\mathbf{w};\mathbf{X},\mathbf{Y})) & = \log \left(\prod_{i = 1}^{n} \theta(\mathbf{w}^T\mathbf{x}_i)^{y_i}\theta(-\mathbf{w}^T\mathbf{x}_i)^{1-y_i}\right) \\
& = \sum_{i = 1}^{n} y_i\log(\theta(\mathbf{w}^T\mathbf{x}_i)) + (1-y_i)\log(\theta(-\mathbf{w}^T\mathbf{x}_i))
\end{align*}
$$

Substituting the logistic function into this equation further simplifies it:

$$\log(\mathcal{L}(\mathbf{w};\mathbf{X},\mathbf{Y})) = \sum_{i = 1}^{n} y_i\log \left(\frac{e^{\mathbf{w}^T\mathbf{x}_i}}{1+ e^{\mathbf{w}^T\mathbf{x}_i}}\right) + (1-y_i)\log \left(\frac{1}{1 + e^{\mathbf{w}^T\mathbf{x}_i}}\right)$$

Typically in optimization, we don't maximize this log-likelihood directly. Instead, we minimize the negative of this quantity, which is called the log-loss or cross-entropy loss function. This function serves as our objective or cost function during training:

$$E(\mathbf{w};\mathbf{X},\mathbf{Y}) = -\log(\mathcal{L}(\mathbf{w};\mathbf{X},\mathbf{Y})) = -\sum_{i = 1}^{n} y_i\log \left(\frac{e^{\mathbf{w}^T\mathbf{x}_i}}{1+ e^{\mathbf{w}^T\mathbf{x}_i}}\right) - (1-y_i)\log \left(\frac{1}{1 + e^{\mathbf{w}^T\mathbf{x}_i}}\right)$$

This cross-entropy loss comes directly from the negative log-likelihood of the Bernoulli distribution, and this is why it's appropriate for binary classification problems like this one. It measures the dissimilarity between the ground-truth labels and the predicted probabilities. The better our predictions, the lower the cross-entropy loss.

To obtain the optimal weights vector $\mathbf{w}$, we would typically differentiate the log-likelihood function with respect to $\mathbf{w}$, equate the result to zero, and then solve for $\mathbf{w}$. However, unlike linear regression, the log-likelihood formulation lacks a closed form solution for the weight vector. Instead, we use methods like gradient ascent due to the concavity of the log-likelihood function, which ensures a unique global maximum. After calculating the optimal weights, we can use the model equation to make prediction:


$$P(\mathbf{Y}|\mathbf{X} = \mathbf{x}_i) = \theta(\mathbf{w}^T\mathbf{x}_i)^{Y}\theta(-\mathbf{w}^T\mathbf{x}_i)^{1-Y}$$



In [246]:
model = LogisticRegression(solver='liblinear', random_state=1)
model.fit(X_train, y_train)

LogisticRegression(random_state=1, solver='liblinear')

In [247]:
val_dict = df_val[categorical + numerical].to_dict(orient='records')
X_val = dv.transform(val_dict)
y_pred = model.predict_proba(X_val)

The first column of the array contains the probability that the target is negative
(no churn), and the second column contains the probability that the target is positive
(churn).

To make the actual decision about whether to send a promotional letter to our customers,
using the probability alone is not enough. We need hard predictions — binary
values of True (churn, so send the mail) or False (not churn, so don’t send the mail). To get the binary predictions, we take the probabilities and cut them above a certain
threshold. If the probability for a customer is higher than this threshold, we predict
churn, otherwise, not churn. If we select 0.5 to be this threshold, making the
binary predictions is easy.

In [248]:
p = y_pred[:,1]
churn = p > 0.5
printest('prediction of clients that churn:',churn)
printest('clients that churn:',y_val)



prediction of clients that churn: : 
 [False False False ... False  True False] 

clients that churn: : 
 [0 1 0 ... 0 0 0] 



In [249]:
# True if our prediction is correct and False if it’s not.
printest('predictions', y_val == churn)
accuracy = (y_val == churn).mean()
printest('accuracy', accuracy)

predictions : 
 [ True False  True ...  True False  True] 

accuracy : 
 0.8016129032258065 



The result from the mean is the fraction of ones in that array, which we already used
for calculating the churn rate. Because “1” (True) in this case is a correct prediction and “0” (False) is an incorrect prediction, the resulting number tells us the percentage of correct predictions.

An accuracy of 0.80 means that the model
predictions matched the actual value 80% of the time, or the model makes correct
predictions in 80% of cases.

# Model interpretation

For logistic regression the model learn from the overall data a vector of weights $\mathbf{w} = (w_1, \cdots, w_n)$ and the bias term $w_0$. Let's get the vector of weights and correlate with the respective features they represent.

In [250]:
feature_names = dv.get_feature_names_out()
weights = model.coef_[0]

dict(zip(feature_names,weights.round(3)))

{'contract=month-to-month': 0.563,
 'contract=one_year': -0.086,
 'contract=two_year': -0.599,
 'dependents=no': -0.03,
 'dependents=yes': -0.092,
 'deviceprotection=no': 0.1,
 'deviceprotection=no_internet_service': -0.116,
 'deviceprotection=yes': -0.106,
 'gender=female': -0.027,
 'gender=male': -0.095,
 'internetservice=dsl': -0.323,
 'internetservice=fiber_optic': 0.317,
 'internetservice=no': -0.116,
 'monthlycharges': 0.001,
 'multiplelines=no': -0.168,
 'multiplelines=no_phone_service': 0.127,
 'multiplelines=yes': -0.081,
 'onlinebackup=no': 0.136,
 'onlinebackup=no_internet_service': -0.116,
 'onlinebackup=yes': -0.142,
 'onlinesecurity=no': 0.258,
 'onlinesecurity=no_internet_service': -0.116,
 'onlinesecurity=yes': -0.264,
 'paperlessbilling=no': -0.213,
 'paperlessbilling=yes': 0.091,
 'partner=no': -0.048,
 'partner=yes': -0.074,
 'paymentmethod=bank_transfer_(automatic)': -0.027,
 'paymentmethod=credit_card_(automatic)': -0.136,
 'paymentmethod=electronic_check': 0.175,


Let's use a small set of features, contract (categorical), tenure (numeric) and totalcharges(numeric). For the contract features is necessary to use one hot encoder as follows:

In [251]:
subset = ['contract', 'tenure', 'totalcharges']
train_dict_small = df_train[subset].to_dict(orient = 'records')

dv_small = DictVectorizer(sparse=False)
dv_small.fit(train_dict_small)
X_small_train = dv_small.transform(train_dict_small)

model_small = LogisticRegression(solver='liblinear', random_state=1)
model_small.fit(X_small_train, y_train)

printest('Bias:',model_small.intercept_[0])
printest('Weights',dict(zip(dv_small.get_feature_names_out(),model.coef_[0].round(3))))

Bias: : 
 -0.577229912199359 

Weights : 
 {'contract=month-to-month': 0.563, 'contract=one_year': -0.086, 'contract=two_year': -0.599, 'tenure': -0.03, 'totalcharges': -0.092} 



| Bias| Contract month|Contract year|Contract 2 year| tenure | charges|
| --- | --- | --- |--- |--- |--- |
| $ -0.577$ | $0.563$ | $-0.086$ |  $-0.599$ |  $-0.03$ | $-0.092 $|

In linear regression, the bias term denotes the model's baseline prediction when there's no information about the other variables. In logistic regression, the model's interpretation becomes more complex due to the implementation of the sigmoid function before producing the final output. Let's illustrate this with the features: contract month ($X_{1}$), contract year ($X_{2}$), contract for 2 years ($X_{3}$), tenure ($X_{4}$), and charges ($X_{5}$).

We can represent these as a feature vector, with ${X}_{0}$ set to one for the bias:

$$\mathbf{X} = ({X}_{0}, {X}_{1},{X}_{2},{X}_{3}, {X}_{4}, {X}_{5})^T$$

Similarly, let's denote the weights as $\mathbf{w} = (w_0,w_1,w_2,w_3,w_4,w_5)$. In linear regression, the predicted target variable $\mathbf{\hat{y}}$ for $\mathbf{X}$ is given by

$$\mathbf{\hat{y}} = \mathbf{w}^T\mathbf{X} $$

For each individual customer, we could write this as

$$\mathbf{\hat{y}}_i = \mathbf{w}^T\mathbf{x}_i.$$

Now, for logistic regression, we interpret the weights differently. After applying the sigmoid function, we obtain:

$$P(\mathbf{Y} = 1|\mathbf{X} = \mathbf{x}_i) = \theta(\mathbf{\hat{y}}_i)= \frac{e^{\mathbf{w}^T\mathbf{x}_i}}{1+ e^{\mathbf{w}^T\mathbf{x}_i}}$$

This calculates the probability of the $i^{th}$ customer churning. The bias term signifies the baseline prediction of the model. If we set all the feature values to zero, except for $x_0 = 1$ which corresponds to the bias, we derive:
$$P(\mathbf{Y} = 1|\mathbf{X} = x_0 = 1) = \frac{e^{w_0}}{1+ e^{w_0}}$$

In our specific problem, we have $w_0 = -0.577$, yielding a churn probability of $Pr = 0.36 $. This suggests that, on average, a customer is more likely to stay with the service than to leave it.

# Using the model

We applied the model to the validation set, computed the probabilities
of churning for every customer there, and concluded that the model is 80% accurate.

In [252]:
accuracy = (y_val == churn).mean()
printest('accuracy', accuracy)

accuracy : 
 0.8016129032258065 



To evaluate the model we can take a random customer from the test dataset as follows:

In [261]:
#Transform dataframe in dict inside a list
test_dict = df_test[categorical + numerical].to_dict(orient='records')
i = int(np.random.randint(df_test.shape[0], size =1))
customer = test_dict[i]
X_test = dv.transform([customer])

y_pred = model.predict_proba(X_test)
printest('Prob Not churn| Prob Churn',y_pred)

Prob Not churn| Prob Churn : 
 [[0.69187648 0.30812352]] 

